# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [ ]:
! pip install ipywidgets
# Run the below in GitBash
# jupyter nbextension enable --py gmaps
# jupyter nbextension enable --py widgetsnbextension 
# or
# jupyter nbextension enable --py widgetsnbextension --sys-prefix

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import csv
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
csvpath = os.path.join("ExportCitiesfile.csv")
export_cities_file = "ExportCitiesfile.csv"
cities_dataframe = pd.read_csv(export_cities_file)
cities_dataframe

,Unnamed: 0,Country,City,Latitude,Longitude,Date,Max Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph)
0,0,AR,san luis,-33.2950,-66.3356,1647747113,58.98,44,59,11.65
1,1,MA,agadir,30.4202,-9.5982,1647747113,59.45,87,56,4.43
2,2,GF,sinnamary,5.3833,-52.9500,1647747114,76.33,89,100,11.97
3,3,GB,brae,60.3964,-1.3530,1647747114,44.74,69,62,25.75
4,4,AU,new norfolk,-42.7826,147.0587,1647747115,68.92,52,100,1.01
...,...,...,...,...,...,...,...,...,...,...
563,563,UA,markova,48.6758,24.4138,1647747270,22.69,64,1,3.18
564,564,AU,victor harbor,-35.5667,138.6167,1647747271,68.00,77,94,9.73
565,565,US,meadow lake,34.8014,-106.5436,1647747271,57.34,11,98,8.86
566,566,US,salina,38.8403,-97.6114,1647747271,50.95,52,0,5.75


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [4]:
# Store 'Lat' and 'Lng' into  locations 
locations = cities_dataframe[["Latitude", "Longitude"]].astype(float)
# locations

# Convert Humidity to float and store to use as the weight
Humidity = cities_dataframe["Humidity (%)"].astype(float)
# Humidity

In [5]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=Humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
# A max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 10 mph.
# Zero cloudiness.

# Drop any rows that don't satisfy all three conditions. You want to be sure the weather is ideal.

# Create new DataFrame fitting weather criteria
Weather_criteria = cities_dataframe.loc[(cities_dataframe["Max Temperature (F)"]<80) & (cities_dataframe["Max Temperature (F)"]>70) &(cities_dataframe["Wind Speed (mph)"]<10) & (cities_dataframe["Cloudiness (%)"]==0), :]

# Drop any rows will null values
Weather_criteria_clean = Weather_criteria.dropna()
del Weather_criteria_clean["Unnamed: 0"]
Weather_criteria_clean_reset = Weather_criteria_clean.reset_index()

Weather_criteria_clean_reset

,index,Country,City,Latitude,Longitude,Date,Max Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph)
0,63,BR,caravelas,-17.7125,-39.2481,1647747130,73.58,90,0,2.48
1,79,MX,matamoros,25.5333,-103.2500,1647747134,71.71,20,0,4.61
2,106,MX,ojinaga,29.5667,-104.4167,1647747142,70.41,19,0,5.75
3,206,ML,kidal,18.4411,1.4078,1647747171,77.09,15,0,9.42
4,225,PK,barkhan,29.8977,69.5256,1647747176,76.23,23,0,5.32
5,273,PR,cabo rojo,18.0866,-67.1457,1647747189,76.68,81,0,8.79
6,362,BR,vila velha,-20.3297,-40.2925,1647747215,76.95,92,0,3.44
7,363,MX,cabo san lucas,22.8909,-109.9124,1647747215,75.51,48,0,8.05
8,554,PR,hatillo,18.4863,-66.8254,1647747268,75.78,84,0,4.00


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
Hotel_df = Weather_criteria_clean_reset
Hotel_df["Hotel Name"] = ""
Hotel_df

,index,Country,City,Latitude,Longitude,Date,Max Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Hotel Name
0,63,BR,caravelas,-17.7125,-39.2481,1647747130,73.58,90,0,2.48,
1,79,MX,matamoros,25.5333,-103.2500,1647747134,71.71,20,0,4.61,
2,106,MX,ojinaga,29.5667,-104.4167,1647747142,70.41,19,0,5.75,
3,206,ML,kidal,18.4411,1.4078,1647747171,77.09,15,0,9.42,
4,225,PK,barkhan,29.8977,69.5256,1647747176,76.23,23,0,5.32,
5,273,PR,cabo rojo,18.0866,-67.1457,1647747189,76.68,81,0,8.79,
6,362,BR,vila velha,-20.3297,-40.2925,1647747215,76.95,92,0,3.44,
7,363,MX,cabo san lucas,22.8909,-109.9124,1647747215,75.51,48,0,8.05,
8,554,PR,hatillo,18.4863,-66.8254,1647747268,75.78,84,0,4.00,


In [10]:
# url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json" 
# places_data = requests.get(url, params=params).json()
# print(json.dumps(places_data, indent=4, sort_keys=True))

In [9]:
# Use Google Places API to find the first hotel for each city located within 5,000 meters of your coordinates.
for index, row in Hotel_df.iterrows():
    try:
        url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

        params = {"location": "location", 
                  "keyword": "hotel", 
                  "radius": 5000, 
                  "key": g_key}


        Latitude = row["Latitude"]
        Longitude = row["Longitude"]

        params["location"] = f"{Latitude}, {Longitude}"

        Hotel_data = requests.get(url, params=params).json()
        
        Hotel_df.loc[index, "Hotel Name"] = Hotel_data["results"][0]["name"]
        
    except IndexError:
        
        Hotel_df.loc[index, "Hotel Name"] = "NaN"
        
Hotel_df

,index,Country,City,Latitude,Longitude,Date,Max Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Hotel Name
0,63,BR,caravelas,-17.7125,-39.2481,1647747130,73.58,90,0,2.48,Pousada Liberdade
1,79,MX,matamoros,25.5333,-103.2500,1647747134,71.71,20,0,4.61,HOTEL PLAZA MATAMOROS
2,106,MX,ojinaga,29.5667,-104.4167,1647747142,70.41,19,0,5.75,Hotel Marval
3,206,ML,kidal,18.4411,1.4078,1647747171,77.09,15,0,9.42,NaN
4,225,PK,barkhan,29.8977,69.5256,1647747176,76.23,23,0,5.32,Noor Khan Hotel
5,273,PR,cabo rojo,18.0866,-67.1457,1647747189,76.68,81,0,8.79,Hotel Mi Tierra
6,362,BR,vila velha,-20.3297,-40.2925,1647747215,76.95,92,0,3.44,Hotel Golden Tulip Porto Vitória
7,363,MX,cabo san lucas,22.8909,-109.9124,1647747215,75.51,48,0,8.05,Medano Hotel and Suites
8,554,PR,hatillo,18.4863,-66.8254,1647747268,75.78,84,0,4.00,El Buen Hotel


In [14]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
Hotel_info = [info_box_template.format(**row) for index, row in Hotel_df.iterrows()]
locations = Hotel_df[["Latitude", "Longitude"]]

In [16]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=Hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))